In [5]:
import re
import json
import requests
from bs4 import BeautifulSoup

# City borders

In [72]:
def extract_json_from_js(text: str):
    """
    Find all JSON.parse('...') calls in a JS block, decodes the JSON, and yields only those where type == 'FeatureCollection'.

    Args:
        text (str): JS text.

    Returns:
        data (dict): GeoJSON object.
    """
    pattern = re.compile(r'JSON\.parse\(\s*["\'](?P<json>\{.*?\})["\']\s*\)', re.DOTALL)

    for match in pattern.finditer(text):
        json_text = match.group("json")
        json_text = json_text.encode("utf-8").decode("unicode_escape")

        try:
            data = json.loads(json_text)
            if isinstance(data, dict) and data.get("type") == "FeatureCollection":
                yield data
        except json.JSONDecodeError:
            continue

In [20]:
url = "https://parkingreform.org/parking-lot-map/"

In [21]:
response = requests.get(url)
response.raise_for_status()
html = response.text

In [22]:
soup = BeautifulSoup(html, "html.parser")
scripts = soup.find_all("script")

In [67]:
city_border_geojson = []
for script in scripts:
    for data in extract_json_from_js(script.text):
        city_border_geojson.append(data)

In [ ]:
def flatten_coords(coords):
    """
    Flatten geographic coordinates from X,Y,Z to X,Y.

    Args:
        coords (list): a list of X,Y,Z coordinates. May include multiple nested lists.

    Returns:
        list: A list of flattened X,Y coordinates.
    """

    if isinstance(coords[0], (float, int)):
        return coords[:2]
    
    else:
        return [flatten_coords(c) for c in coords]

In [ ]:
def flatten_geojson(geojson: dict):
    """
    Take a GeoJSON dict and flatten all coordinates objects.

    Args:
        geojson (dict): a GeoJSON dict.
    
    Returns:
        geojson (dict): the same GeoJSON dict with all coordinates flattened from X,Y,Z to X,Y.
    """

    if geojson["type"] in ["Polygon", "MultiPolygon", "LineString", "MultiLineString", "Point", "MultiPoint"]:
        geojson["coordinates"] = flatten_coords(geojson["coordinates"])

    elif geojson["type"] == "Feature":
            flatten_geojson(geojson["geometry"])

    elif geojson["type"] == "FeatureCollection":
        for feature in geojson["features"]:
            flatten_geojson(feature)

    return geojson

In [ ]:
flat_geojson = flatten_geojson(city_border_geojson[0])

# Parking lots